# 데이터 셋 만들기

In [1]:
from IPython.display import display
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import random

import torchvision.transforms as transforms

import os

from matplotlib.pyplot import imshow

from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import matplotlib.image as img
from sklearn.model_selection import train_test_split

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)
random.seed(0)


if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
class CustomDataset(Dataset):
  def __init__(self, imgs, malices, names):
        super().__init__()
        self.imgs = imgs
        self.malices = malices
        self.names = names
        
  def __len__(self):
      return len(self.imgs)
  
  def __getitem__(self,index):
      return self.imgs[index], self.malices[index], self.names[index]

In [4]:
from zipfile import ZipFile
from PIL import Image
import zipfile
import cv2

import random

def convertPILImageToCV2(pil_image):
  np_image = np.array(pil_image)
  open_cv_image = np.array(pil_image) 

  open_cv_image = open_cv_image[:, :, ::-1].copy() 

  return open_cv_image

benign_path = '/content/drive/MyDrive/Malware Analysis/kisa_dataset/benign_image.zip'
malign_path = '/content/drive/MyDrive/Malware Analysis/kisa_dataset/malign_image.zip'
benign_zip = ZipFile(benign_path,'r')
malign_zip = ZipFile(malign_path,'r')

#benign
benign_info_list = benign_zip.infolist()
benign_name_list = benign_zip.namelist()
malign_info_list = malign_zip.infolist()
malign_name_list = malign_zip.namelist()

benign_set = []
malign_set = []



for i in range(len(benign_info_list)):
  _file = benign_zip.open(benign_info_list[i])
  img = Image.open(_file)
  img = convertPILImageToCV2(img)
  name = benign_name_list[i]
  benign_set.append((img, 0, name))
  

for i in range(len(malign_info_list)):
  _file = malign_zip.open(malign_info_list[i])
  img = Image.open(_file)
  img = convertPILImageToCV2(img)
  name = malign_name_list[i]
  malign_set.append((img, 1, name))
  

num_each_test = 400

raw_test_set = benign_set[:num_each_test] + malign_set[:num_each_test]
# random.shuffle(raw_test_set)
raw_train_set = benign_set[num_each_test:] + malign_set[num_each_test:]
# random.shuffle(raw_train_set)


## Normalizing

In [5]:
train_meanRGB = [np.mean(x, axis=(1,2)) for x,_, __ in raw_train_set]
train_stdRGB = [np.std(x, axis=(1,2)) for x,_, __ in raw_train_set]

train_meanR = np.mean([m[0] for m in train_meanRGB])
train_meanG = np.mean([m[1] for m in train_meanRGB])
train_meanB = np.mean([m[2] for m in train_meanRGB])

train_stdR = np.mean([s[0] for s in train_stdRGB])
train_stdG = np.mean([s[1] for s in train_stdRGB])
train_stdB = np.mean([s[2] for s in train_stdRGB])

test_meanRGB = [np.mean(x, axis=(1,2)) for x,_, __ in raw_test_set]
test_stdRGB = [np.std(x, axis=(1,2)) for x,_, __ in raw_test_set]

test_meanR = np.mean([m[0] for m in test_meanRGB])
test_meanG = np.mean([m[1] for m in test_meanRGB])
test_meanB = np.mean([m[2] for m in test_meanRGB])

test_stdR = np.mean([s[0] for s in test_stdRGB])
test_stdG = np.mean([s[1] for s in test_stdRGB])
test_stdB = np.mean([s[2] for s in test_stdRGB])



ang = (0-train_meanR)/train_meanR
print(ang)
normalized_zero_r = (0-test_meanR)/test_stdR
print(normalized_zero_r)


-1.0
-0.10260687918659961


In [6]:
train_transform = transforms.Compose([transforms.ToPILImage(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([train_meanR,train_meanG,train_meanB],[train_stdR, train_stdG, train_stdB])
                                      ])
test_transform = transforms.Compose([transforms.ToPILImage(),
                                      transforms.ToTensor(),
                                     transforms.Normalize([test_meanR, test_meanG, test_meanB],[test_stdR, test_stdG, test_stdB])
                                      ])

train_imgs, train_malices, train_names = [],[], []
test_imgs, test_malices, test_names = [],[],[]


for img,mal,name in raw_train_set:
  train_imgs.append(train_transform(img))
  train_malices.append(mal)
  train_names.append(name)
for img,mal,name in raw_test_set:
  test_imgs.append(test_transform(img))
  test_malices.append(mal)
  test_names.append(name)


train_set = CustomDataset(train_imgs, train_malices, train_names)
test_set = CustomDataset(test_imgs, test_malices, test_names)

# 모델 정의

In [7]:
%%time
class CNN(nn.Module): 
    def __init__(self):
      super(CNN, self).__init__()
      
      self.layer1 = nn.Sequential(
          # kerner size : 필터 크기
          nn.Conv2d(in_channels=3, out_channels=28, kernel_size=5),
          nn.BatchNorm2d(28),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(2),
          nn.Dropout(0.4)
      ) # output size : (이미지 width - 커널사이즈 + 2*패딩사이즈)/stride + 1 -> 
      # (256 - 5 + 2* 0)/1 + 1 252 -> 126 126(풀링)
      # (128 - 5) + 1 -> 124 -> 62 (풀링)
      # (64 - 5 + 2* 0)/1 + 1 -> 60 -> 30 (풀링)
      # 
      self.layer2 = nn.Sequential(
          nn.Conv2d(in_channels=28, out_channels=23, kernel_size=5),
          nn.BatchNorm2d(23),
          nn.ReLU(inplace=True),
          nn.MaxPool2d(2),
          nn.Dropout(0.4)
      )
      # 

      # 256 인경우 input : 126 * 126, 128인 경우 30*30
      # (126-5)/1+1 : 122 -> 61
      # (62 - 5) + 1 : 58 -> 풀링하면 29
      # (30-5) +1 -> 26, 풀링하면 13
      # 126 * 126 * 10 --CONV2--> 122 * 122 * 20 --MAX POOL--> 61 * 61 * 20

      ## 60 60 
      
      # 256 256 일때 39 * 61 * 61, out_features = 183
      self.layer3 = nn.Sequential(
          nn.Linear(in_features=23 * 61 * 61, out_features=421, bias=True),
          nn.BatchNorm1d(421),
          nn.ReLU(inplace=True),
          nn.Dropout(0.4)
      )

      self.layer4 = nn.Sequential(
          nn.Linear(in_features=421, out_features=227, bias=True),
          nn.ReLU(inplace=True),
          nn.Dropout(0.4)
      )

      self.layer5 = nn.Sequential(
          nn.Linear(in_features=227, out_features=2, bias=True),
      )

    def forward(self, x):
      x = self.layer1(x)
      x = self.layer2(x)
      x = x.view(-1, 23 * 61 * 61)
      x = self.layer3(x)
      x = self.layer4(x)
      x = self.layer5(x)
      return x

CPU times: user 25 µs, sys: 4 µs, total: 29 µs
Wall time: 32.4 µs


In [8]:
train_losses = []
valid_losses = []

num_epochs = 93
batch_size = 256
learning_rate = 0.002329
weight_decay = 0.0003281

train_loader = DataLoader(dataset = train_set, batch_size = batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(dataset = test_set, batch_size = batch_size, shuffle=False, num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CNN().to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate, weight_decay=weight_decay)

print(device)
print(model)



cuda
CNN(
  (layer1): Sequential(
    (0): Conv2d(3, 28, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.4, inplace=False)
  )
  (layer2): Sequential(
    (0): Conv2d(28, 23, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(23, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.4, inplace=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=85583, out_features=421, bias=True)
    (1): BatchNorm1d(421, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.4, inplace=False)
  )
  (layer4): Sequential(
    (0): Linear(in_features=421, out_features=227, bias=True)
  

# Training 

In [9]:
# torch.cuda.empty_cache()

In [10]:
# keeping-track-of-losses 
# %%time 
import time

  # 현재시각 - 시작시간 = 실행 시간
for epoch in range(1, num_epochs + 1):
    # keep-track-of-training-and-validation-loss
    train_loss = 0.0
    valid_loss = 0.0
    
    # training-the-model
    model.train()
    
    start = time.time()
    for data, target,_ in train_loader:
        
        # move-tensors-to-GPU 
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        
    # validate-the-model
    model.eval()
    correct = 0
    total = 0
        
    for data, target,_ in test_loader:
        
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()
              
        loss = criterion(output, target)
        # update-average-validation-loss 
        valid_loss += loss.item() * data.size(0)
    
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(test_loader.sampler)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    # print-training/validation-statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    print("time :", time.time() - start)
    print('Test Accuracy of the model: {} %'.format(100 * correct / total))

    # # test-the-model
    # with torch.no_grad():
    #     correct = 0
    #     total = 0
    #     for images, labels,_ in test_loader:
    #         images = images.to(device)
    #         labels = labels.to(device)
    #         outputs = model(images)
    #         _, predicted = torch.max(outputs.data, 1)
    #         total += labels.size(0)
    #         correct += (predicted == labels).sum().item()
              
    #     print('Test Accuracy of the model: {} %'.format(100 * correct / total))

Epoch: 1 	Training Loss: 0.521583 	Validation Loss: 1.624495
time : 4.767258882522583
Test Accuracy of the model: 50.0 %
Epoch: 2 	Training Loss: 0.361944 	Validation Loss: 0.678951
time : 4.218108415603638
Test Accuracy of the model: 55.125 %


RuntimeError: ignored

# Testing

In [ ]:
# test-the-model
model.eval()  # it-disables-dropout
predict = []
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels,_ in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        predict.extend(predicted.detach().cpu())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
          
    print('Test Accuracy of the model: {} %'.format(100 * correct / total))

# Save 
torch.save(model.state_dict(), 'model.ckpt')

In [ ]:
#predict
test_loader

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

plt.plot(train_losses, label='Training loss')
plt.plot(valid_losses, label='test loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(frameon=False)

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
t = confusion_matrix(test_set.malices, predict)
f1 = f1_score(test_set.malices, predict)
print(t)
print(f'f1 score : {f1}')

np_predict = np.array(predict)
np_malices = np.array(test_set.malices)


t = np_predict == np_malices # T : 정답이 맞는 경우 
f = np_predict != np_malices # F : 정답에 틀린 경우

# 각 그룹의 베이직 블록 개수, 픽셀 개수, (베이직 블록 - 픽셀 개수), 그룹당 평균, 중간
index_true = np.where(t)[0]
index_false = np.where(f)[0]
benign_block_dir = '/content/drive/MyDrive/Malware Analysis/kisa_dataset/benign_block/'
malign_block_dir = '/content/drive/MyDrive/Malware Analysis/kisa_dataset/malign_block/'

In [ ]:
import torch
import math


E = 0.0000001

def truncate(num, n):
    integer = int(num * (10**n))/(10**n)
    return float(integer)

def get_num_pixel(tensor_r, normalized_zero_r):
  pixels = (tensor_r.numpy() - normalized_zero_r) > E 
  cnt = np.count_nonzero(pixels)
  return cnt

true_benign_basic_block_nums = []
true_malign_basic_block_nums = []
true_benign_pixel_nums = []
true_malign_pixel_nums = []

false_benign_basic_block_nums = []
false_malign_basic_block_nums = []
false_benign_pixel_nums = []
false_malign_pixel_nums = []

normalized_zero_r = (0-test_meanR)/test_stdR
normalized_zero_r = truncate(normalized_zero_r,8)

demo = 400 # 최대 400까지 

for idx in index_true[:demo]:
  tensor, malice, name = test_set[idx]
  name = name.replace('.png','.txt')
  path = benign_block_dir+name if malice == 0 else malign_block_dir+name
  # 베이직 블록 개수 세기 위해, 파일을 엽니다. 
  block_file = open(path,'r')
  num_block = len(block_file.readlines())
  num_pixels = get_num_pixel(tensor[0], normalized_zero_r)
  
  # 베이직 블록 개수, 픽셀 개수를 입력합니다. 
  if malice == 0:
    true_benign_basic_block_nums.append(num_block)
    true_benign_pixel_nums.append(num_pixels)
    
  else: 
    true_malign_basic_block_nums.append(num_block)
    true_malign_pixel_nums.append(num_pixels)

print()
for idx in index_false[:demo]:
  tensor, malice, name = test_set[idx]
  name = name.replace('.png','.txt')
  path = benign_block_dir+name if malice == 0 else malign_block_dir+name
  # 베이직 블록 개수 세기 위해, 파일을 엽니다. 
  block_file = open(path,'r')
  num_block = len(block_file.readlines())
  num_pixels = get_num_pixel(tensor[0], normalized_zero_r)
  
  # 베이직 블록 개수, 픽셀 개수를 입력합니다. 
  if malice == 0:
    false_benign_basic_block_nums.append(num_block)
    false_benign_pixel_nums.append(num_pixels)
  else: 
    false_malign_basic_block_nums.append(num_block)
    false_malign_pixel_nums.append(num_pixels)

  

In [ ]:
# True_benign
# True_malign
# False_benign
# False_malign
def stat(block_nums, pixel_nums, label):
  min_block = min(block_nums)
  mean_block = np.mean(block_nums)
  min_pixels = min(pixel_nums)
  mean_pixels = np.mean(pixel_nums)
  diff = np.array(block_nums) - np.array(pixel_nums)
  min_diff = min(diff)
  mean_diff = np.mean(diff)

  median_block = np.median(block_nums)
  median_pixels = np.median(pixel_nums)
  
  print(f'====={label}=====')
  print(f'블록 개수 평균 : {mean_block}, 픽셀 개수 평균 : {mean_pixels}, 평균 차이 : {mean_diff} 정보 손실률: {100 - (mean_pixels/mean_block) * 100}%')
  print(f'중간값 블록 : {median_block}, 중간값 픽셀 : {median_pixels}')

stat(true_benign_basic_block_nums, true_benign_pixel_nums, "True benign")
stat(true_malign_basic_block_nums, true_malign_pixel_nums, 'True malign')
stat(false_benign_basic_block_nums, false_benign_pixel_nums, 'False benign')
stat(false_malign_basic_block_nums, false_malign_pixel_nums, 'False malign')

In [ ]:
# 잘 구분되는 benign, malign


In [ ]:
import torch
import torchvision
import torchvision.transforms as T
from PIL import Image

import matplotlib.pyplot as plt
import torchvision.transforms as transforms


indecies = np.where(t)[0]

cnt = 0

for i in indecies:
  # wandb.log({"fakge": [wandb.Image(i.reshape([256, 256, 3]), mode="RGB") for i in test_set[i][0]]})
  tf = T.ToPILImage()
  img_t = tf(raw_test_set[i][0])
  plt.imshow(img_t)
  plt.show()
  print('ang')
  
  cnt += 1
  if cnt > 20 :
    break


In [ ]:
indecies = np.where(f)[0]

cnt = 0

for i in indecies:
  # wandb.log({"fakge": [wandb.Image(i.reshape([256, 256, 3]), mode="RGB") for i in test_set[i][0]]})
  tf = T.ToPILImage()
  img_t = tf(raw_test_set[i][0])
  plt.imshow(img_t)
  plt.show()
  print('ang')
  
  cnt += 1
  if cnt > 20 :
    break
